In [9]:
import readdata
from text_vectorizer import CV
from text_vectorizer import TFIDF
from text_vectorizer import word2vec
from text_vectorizer import outlierDection
from OutlierDetectRemove import removeOutliers
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import GridSearchCV
import numpy as np
import sys

# Usage: 
# 1. python lstm.py cv
# 2. python lstm.py tfidf
# 3. python lstm.py word2vec

# cross-validation 3
# number of estimators: 200, 400, 800
# max_depth: 1, 5, 9
# min_samples_leaf: 2, 4
# min_samples_split: 5, 10

def getRemovedVals(X,Y = None,Ftype = "",isTest = False):

    X = np.array(X)
    index,_ = outlierDection(X,Ftype)
    if not isTest:
        Y = np.array(Y)
        Xrem,Yrem = removeOutliers(index,X,Y,Ftype)
        return Xrem,Yrem

    else:
        Xrem = removeOutliers(index,X,Y,Ftype)
        return Xrem


def evaluate(pred, truth):
    print('Mean Absolute Error:', metrics.mean_absolute_error(truth, pred))
    print('Mean Squared Error:', metrics.mean_squared_error(truth, pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(truth, pred)))


dfTrain = readdata.read_clean_data(readdata.TRAINFILEPATH,nolabel = False)
dfTest = readdata.read_clean_data(readdata.TESTFILEPATH,nolabel = True)

X_train = dfTrain['text'].to_numpy()
Y_train = dfTrain['label'].to_numpy()
X_test = dfTest['text'].to_numpy() # unlabelled


# X_train, X_test, _ = CV(X_train, X_test) # train shape: (17973, 141221)
# X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "CV_Train",isTest = False)
# X_test = getRemovedVals(X = X_test,Y = None,Ftype = "CV_Test",isTest = True)


# X_train, X_test, _ = TFIDF(X_train, X_test) # shape: (17973, 141221)
# X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "TFIDF_Train",isTest = False)
# X_test = getRemovedVals(X = X_test,Y = None,Ftype = "TFIDF_Test",isTest = True)


X_train, X_test = word2vec(X_train, X_test)
X_train,Y_train = getRemovedVals(X = X_train,Y = Y_train,Ftype = "W2V_Train",isTest = False)
X_test = getRemovedVals(X = X_test,Y = None,Ftype = "W2V_Test",isTest = True)


# below are the hyperparameters to be grid-searched on
n_estimators = [200, 400, 800]
max_depth = [1, 5, 9]
min_samples_leaf = [2, 4]
min_samples_split = [5, 10]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth,
                min_samples_leaf=min_samples_leaf,
                min_samples_split=min_samples_split)

model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

ModuleNotFoundError: No module named 'readdata'

In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.885997 using {'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 800}


In [1]:
from sklearn.metrics import roc_auc_score

In [3]:
roc_auc_score(Y_train, grid_result.predict_proba(X_train)[:,1])

NameError: name 'Y_train' is not defined

ModuleNotFoundError: No module named 'readdata'